In [ ]:
!git clone https://github.com/andrii-harbour/magic-annotations-colab.git
%cd magic-annotations-colab

!pip install pdf2image==1.14.0
!pip install PyPDF2==3.0.0
!pip install PyMuPDF
!pip install opencv-python
!pip install pytesseract
!apt-get install poppler-utils
!apt-get install tesseract-ocr -y

In [ ]:
"""Main api"""
import json
from io import BytesIO
from google.colab import files

from flask import current_app
from PyPDF2 import PdfReader

from app.debug import create_pdf_with_detections
from app.logic import extract_elements_cv
from app.logic.pdf_utils import extract_form


def detect():
    current_app.logger.info('--- Detecting started ---')

    uploaded = files.upload()

    bytes_file = BytesIO(list(uploaded.values())[0])

    pdf_file = PdfReader(bytes_file)

    current_app.logger.info('--- Binary extracting finished ---')
    cv_coordinates = False

    if '/AcroForm' in pdf_file.trailer['/Root']:
        current_app.logger.info('--- Type "pypdf2" ---')
        doc_pages = extract_form(bytes_file)
        current_app.logger.info('--- PyPDF2 searching finished ---')
    else:
        current_app.logger.info('--- Type "cv" ---')
        doc_pages = extract_elements_cv(bytes_file)
        cv_coordinates = True
        current_app.logger.info('--- CV searching finished ---')

    current_app.logger.info('--- Detecting finished ---')

    create_pdf_with_detections(doc_pages, bytes_file, cv_coordinates)

    results = [page.fillable_elements_to_dict() for page in doc_pages]
    
    # save to json 
    with open('output/result.json', 'w') as f:
      json_result = {'results': results}
      json.dump(json_result, f)

detect()
